In [47]:
import pandas as pd
from thefuzz import fuzz

In [48]:
divs = pd.read_csv('section_boundaries.csv', index_col=0)

In [49]:
divs = divs[divs.start_phrase.notna()]

In [50]:
divs

,datecode,type,start_phrase,end_phrase,Unnamed: 5,Unnamed: 6
332,20060920,FOMC1,"If there are no more questions, we’re ready fo...",Thank you. Let me just summarize quickly what ...,NaN,NaN
333,20060920,FOMC2,"Thank you, Mr. Chairman and some participants....",I will be reading the language from page 25 of...,NaN,NaN
334,20061025,FOMC1,"Any other questions? Dave and Karen, thank you...",Okay. Let me ask the Committee’s preference he...,NaN,NaN
335,20061025,FOMC2,Good morning. We’ve already heard Vincent’s re...,I’ll be reading the directive wording and the ...,NaN,NaN
336,20061212,FOMC1,"Are there other questions? If not, we’re ready...",Thank you. Let me just summarize and add a few...,NaN,NaN
...,...,...,...,...,...,...
509,20170920,FOMC1,All in favor? [Chorus of ayes] Any opposed? [N...,"Well, my thanks to everyone for a rich discuss...",NaN,NaN
510,20171101,FOMC1,Thank you. All in favor? [Chorus of ayes] Any ...,Thank you. My thanks to everyone for a thought...,NaN,NaN
511,20171101,FOMC2,"Good morning, everybody. Let’s get started. An...","Thank you, Madam Chair. As you indicated, this...",NaN,NaN
512,20171213,FOMC1,Thank you. All in favor? [Chorus of ayes] I’m ...,"Thank you. Well, my thanks to everyone for a t...",NaN,NaN


In [5]:
new = divs.pivot(index='datecode', columns='type', values=['start_phrase', 'end_phrase']).reset_index()

In [42]:
res_dfs = []
for group in pd.read_csv('../working-csvs/raw_transcripts.csv', index_col=0).groupby('date'):
    datecode = int(group[0].replace('-', '')) 
    if datecode not in divs['datecode'].unique():
        continue
    FOMC1_bounds = [new.loc[new.datecode == datecode].start_phrase.FOMC1.iloc[0], new.loc[new.datecode == datecode].end_phrase.FOMC1.iloc[0]]
    FOMC2_bounds = [new.loc[new.datecode == datecode].start_phrase.FOMC2.iloc[0], new.loc[new.datecode == datecode].end_phrase.FOMC2.iloc[0]]
    sections = [FOMC1_bounds, FOMC2_bounds]
    transc = group[1]
    for i, section in enumerate(sections):
        transc[f'sec{i + 1}startsim'] = transc['content'].apply(lambda x : fuzz.ratio(x[:len(section[0]) + 10], section[0]))
        transc[f'sec{i + 1}endsim'] = transc['content'].apply(lambda x : fuzz.ratio(x[:len(section[1]) + 10], section[1]))
    maxes = transc.idxmax(numeric_only=True)
    FOMC1 = transc.loc[maxes.sec1startsim:maxes.sec1endsim].copy()
    FOMC1['section'] = 1
    FOMC2 = transc.loc[maxes.sec2startsim:maxes.sec2endsim].copy()
    FOMC2['section'] = 2
    result = pd.concat([FOMC1, FOMC2])
    result = result.loc[:, ~result.columns.isin(['sec1startsim', 'sec1endsim', 'sec2startsim', 'sec2endsim'])]
    res_dfs.append(result)

In [43]:
pd.concat(res_dfs).to_csv('../working-csvs/raw_transcripts_sectioned.csv')

In [44]:
df = pd.read_csv('../working-csvs/raw_transcripts_sectioned.csv',index_col=0)